In [8]:
import pandas as pd

df_tags = pd.read_csv('../data/raw/ml-25m/tags.csv')
df_gscores = pd.read_csv('../data/raw/ml-25m/genome-scores.csv')
df_gtags = pd.read_csv('../data/raw/ml-25m/genome-tags.csv')
df_links = pd.read_csv('../data/raw/ml-25m/links.csv')
df_movies = pd.read_csv('../data/raw/ml-25m/movies.csv')
df_ratings = pd.read_csv('../data/raw/ml-25m/ratings.csv')

df_ratings.head(5)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [12]:
##create large dataframe
#The same movieId refers to the same movie between `ratings.csv`, `tags.csv`, `movies.csv`, and `links.csv`
df = pd.merge(df_movies, df_links, how="left", on=['movieId', 'userId'])
df = pd.merge(df, df_tags, on=['movieId'], how='left')
df = pd.merge(df, df_ratings, on=['movieId', 'userId'], how='left')
#display(df.describe())

KeyError: 'userId'

##### Deadlines:
- Report 1. Data viz (5 graphs) + preprocessing. Friday, May 17
- Report 2. Modelling. Friday, June 14
- Final report. Friday, June 21
